**Imports**

In [ ]:
import praw
import pprint
import re
import csv
import pandas as pd
import pprint

**Authentication**

In [ ]:
reddit = praw.Reddit(
    client_id=clientid,
    client_secret=secret,
    password=password,
    user_agent=script,
    username=username,
)

In [ ]:
print(reddit.user.me())

**Functions to retrieve users's gender (implemented from RedDust)**

In [ ]:
#natural language patterns
fem = ["woman", "female", "girl", "lady", "wife", "mother", "sister"]
male = ["man", "male", "boy", "husband", "father", "brother"]

def gender_word(text, gender_list):
    pattern = ["I am a ", "I'm a ", "I am ", "I'm "]
    words = re.sub(r'[^\w\s]', '', text)
    words = words.split()
    for p in pattern:
        if p in text:
            p = re.sub(r'[^\w\s]', '', p)
            p = p.split() 
            for exp in gender_list:
                for i in range(len(words)):
                    if exp == words[i]:
                        idx = words.index(exp)
                        #print(idx)
                        if words.index(p[0]) == idx - 1 or words.index(p[0]) == idx - 2 or words.index(p[0]) == idx -3:
                            return True
    return False

In [ ]:
#patterns used with brackets (e.g. [45f])
fem2 = r"\b\d+f\b"
male2 = r"\b\d+m\b"

def bracket_patterns(text, regex):
    pattern = ["I", "Im", "me", "Me", "my", "My"]
    words = re.sub(r'[^\w\s]', '', text)
    words = words.split()
    #print(words)
    matches = []
    for word in words:
        matches.extend(re.findall(regex, word))
    if matches:
        #print(matches)
        #print(words)
        for p in pattern:
            if p in words:    
                idx = words.index(p)
               # print(idx)
                for match in matches:
                    #print(words.index(match))
                    if words.index(match) == idx+1:
                        return True
    return False

In [ ]:
#test
text = "I am a male. This is a test. I'm a woman."

gender_word(text, male)

In [ ]:
#test
example = "Me [30m] find something."

bracket_patterns(example, male2)

In [ ]:
#function returns whether a text indicates the writer is female (1) or male (0)
def gender_indicator(selftexts, fem, male, fem2, male2):
    for i in range(len(self_texts)):
        if gender_word(self_texts[i], fem) or  bracket_patterns(self_texts[i], fem2):
            return "1" #female
        if gender_word(self_texts[i], male) or bracket_patterns(self_texts[i], male2):
            return "0" #male

**Retrieving users' gender**

In [ ]:
#retrieving author names from hot posts and their comments on the most popular subreddits

#subreddits used: r/funny, r/AskReddit, r/gaming, r/aww (the four most popular subreddits on Reddit as of 6.3.2024)

subreddit = reddit.subreddit("gaming") 
#limiting it to 10 posts
hot_posts = subreddit.hot(limit=10)
authors = []
for post in hot_posts:
    if post.author.name not in authors:
        authors.append(post.author.name)
    comments = post.comments.list()
    #looking for the authors of 10 comments on each of the posts
    for comment in comments[:10]:
        if comment.author != None:
            if comment.author.name not in authors:
                authors.append(comment.author.name)

In [ ]:
len(authors)

In [ ]:
#find out which of the authors indicate gender in their posts

gendered_authors = []
for author in authors:
    user = reddit.redditor(author)
    #try is used because not every author has written a post (some have only written comments)
    try:
        submissions = user.submissions.hot(limit=None)
        counter = 0
        self_texts = []
        for submission in submissions:
            counter += 1
            self_texts.append(submission.selftext)
        #only use authors with more than 2 posts
        if counter > 2:
            #call function to determine gender
            gender = gender_indicator(self_texts, fem, male, fem2, male2)
            if gender:
                gendered_authors.append([author, gender])
    except:
        pass

In [ ]:
#find out which of the authors indicate their gender in their comments

for author in authors:
    user = reddit.redditor(author)
    #try is used because not every author has written a comment
    try:
        comments = user.comments.hot(limit=None)
        counter = 0
        self_texts = []
        for comment in comments:
            counter += 1
            self_texts.append(comment.body)
        #only users with more than 2 comments are considered
        if counter > 2:
            #calling the function to determine gender
            gender = gender_indicator(self_texts, fem, male, fem2, male2)
            if gender:
                gendered_authors.append([author, gender])
    except:
        pass

In [ ]:
len(gendered_authors)

**Manually checking if a user's gender has been inferred correctly**

In [ ]:
#going through every gendered author's posts and comments
print(gendered_authors[12])
user = reddit.redditor(gendered_authors[12][0])
submissions = user.submissions.hot(limit=None)
for submission in submissions:
        print(submission.selftext)
comments = user.comments.hot(limit=None)
for comment in comments:
        print(comment.body)
print("\n")

In [ ]:
#deleting authors that were wrongly inferred
del gendered_authors[9]

In [ ]:
gendered_authors

In [ ]:
#saving the list as a csv file (4 files, one for each subreddit)

gendered_authors4 = "gendered_authors4.csv"

with open(gendered_authors4, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(gendered_authors)